In [10]:
# !pip3 install openai python-dotenv
import os
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

### Setup Client

In [11]:
from openai import OpenAI

client = OpenAI(
    organization='org-FT4xlyvIqg3QOSciNKjbMwYj',
    api_key=os.environ.get("OPENAI_API_KEY"),
)

### Example Speech To Text (Whisper API)
- TODO: Make it stream

In [3]:
%%time
# Entire audio

audio_file= open("assets/Replica1.m4a", "rb")
transcription = client.audio.transcriptions.create(
    model="whisper-1", 
    file=audio_file,
    language='ro'
)
print(transcription.text)

Bună ziua! Aș vrea să fac o programare la dumneavoastră. Mă cam doare moseaua.
CPU times: user 0 ns, sys: 8.31 ms, total: 8.31 ms
Wall time: 882 ms


### Example LLM

In [14]:
# Entire message
chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": "Say this is a test",}],
    model="gpt-3.5-turbo",
)

In [15]:
chat_completion

ChatCompletion(id='chatcmpl-95G3JAJZ5SIaxFBJEzX2Hv6E2TOKa', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', role='assistant', function_call=None, tool_calls=None))], created=1711040373, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_fa89f7a861', usage=CompletionUsage(completion_tokens=5, prompt_tokens=12, total_tokens=17))

In [3]:
%%time
# Chunked text, streamed, closer to IRL
stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

This is a test.CPU times: user 13.2 ms, sys: 971 µs, total: 14.2 ms
Wall time: 2.08 s


### Example TTS

In [50]:
%%time
response = client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="alloy",
    input="""I see skies of blue and clouds of white
             The bright blessed days, the dark sacred nights
             And I think to myself
             What a wonderful world""",
    response_format='opus'
)

CPU times: user 16 µs, sys: 5 µs, total: 21 µs
Wall time: 21.7 µs


In [55]:
dir(response)

['_ResponseContextManager__response',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_protocol',
 '_request_func']

In [49]:
%%time

response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input="Bună ziua, ați sunat la Cabinetul X. Eu sunt asistentul dumneavoastră virtual, cu ce vă pot ajuta?",
    response_format='mp3'
)


CPU times: user 6.01 ms, sys: 0 ns, total: 6.01 ms
Wall time: 1.98 s


In [48]:
import requests
import io

url = "https://api.openai.com/v1/audio/speech"
headers = {
    "Authorization": f'Bearer {os.environ.get("OPENAI_API_KEY")}', 
}

data = {
    "model": "tts-1-hd",
    "input": "Bună ziua, ați sunat la Cabinetul X. Eu sunt asistentul dumneavoastră virtual, cu ce vă pot ajuta?",
    "voice": "alloy",
    "response_format": "mp3",
}

with requests.post(url, headers=headers, json=data, stream=True) as response:
    if response.status_code == 200:
        buffer = io.BytesIO()
        for chunk in response.iter_content(chunk_size=4096):
            print(chunk)
            buffer.write(chunk)
    else:
        print(response.status_code)
        print(response)

429
<Response [429]>


### Example TTS Streamed

In [13]:
import os
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
import os
import requests
from time import time
import pyaudio

In [26]:
pa = pyaudio.PyAudio()
pa.get_default_output_device_info()

OSError: No Default Output Device Available

In [15]:
url = "https://api.openai.com/v1/audio/speech"
headers = {
    "Authorization": f'Bearer {os.getenv("OPENAI_API_KEY")}',
}

data = {
    "model": "tts-1",
    "input": "This is a test",
    "voice": "shimmer",
    "response_format": "wav",
}

In [16]:
response = requests.post(url, headers=headers, json=data, stream=True)

In [22]:
def stream_to_speakers() -> None: 
     import pyaudio 
  
     player_stream = pyaudio.PyAudio().open(format=pyaudio.paInt16, channels=1, rate=24000, output=True) 
  
     start_time = time.time() 
  
     with openai.audio.speech.with_streaming_response.create( 
         model="tts-1", 
         voice="alloy", 
         response_format="pcm",  # similar to WAV, but without a header chunk at the start. 
         input="""I see skies of blue and clouds of white 
                 The bright blessed days, the dark sacred nights 
                 And I think to myself 
                 What a wonderful world""", 
     ) as response: 
         print(f"Time to first byte: {int((time.time() - start_time) * 1000)}ms") 
         for chunk in response.iter_bytes(chunk_size=1024): 
             player_stream.write(chunk) 
  
     print(f"Done in {int((time.time() - start_time) * 1000)}ms.") 

In [24]:
stream_to_speakers()

OSError: [Errno -9996] Invalid output device (no default output device)

### TODO: Initial Dialogue

- How do we create this sort of textual history/context in conversation? Maybe some sort of retrieval?


Plan
- play initial intro
- let the audio play, and feed it to a well prompted LLM
- we should construct (written) replies to the chatbot just to see how it works in a conversation
- after, we can replace the written replies with actual voice replies
- then, if the conversation looks somewhat natural and the calendar appointment has been mocked
- we can transition to checking how an agent could actually retrieve that information, and how it could be stored, and whether we would need to use something like langchain to detect a task and perform an action